# YELP Recommender System
The task is to implement from scratch a system recommending businesses to users. You are free to choose the technique at the basis of the recommender system, as well as the strategy to be used in order to populate the utility matrix.

# Setup


### Colab part

In [1]:
if "google.colab" in str(get_ipython()):
    !pip install pyspark
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [2]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


zsh:1: command not found: kaggle
unzip:  cannot find or open ./data/yelp-dataset.zip, ./data/yelp-dataset.zip.zip or ./data/yelp-dataset.zip.ZIP.


# Spark Session Initialization

In [3]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

In [4]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


23/12/27 13:50:39 WARN Utils: Your hostname, MacBook-Pro-di-Luca.local resolves to a loopback address: 127.0.0.1; using 192.168.1.128 instead (on interface en0)
23/12/27 13:50:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 13:50:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Retrieving RDDs

In [5]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    raw_df.show()
    #return raw_df.sample(frac=1/10, random_state = 0).dropna().rdd
    return raw_df.rdd

raw_review_RDD_tmp = from_json_to_RDD(review_filename)#.sample(True, 1, 0)  #699192 rows #shuffle
raw_user_RDD_tmp = from_json_to_RDD(user_filename)
raw_business_RDD_tmp = from_json_to_RDD(business_filename)
#print(raw_business_RDD.filter(lambda x: x[2]=='bqFG0QJY9jj2m55OqAVHeA').first())

raw_review_RDD_tmp.count()
#6990280

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

23/12/27 13:50:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

6990280

### Sampling User and Reviews RDDs

In [6]:
raw_review_RDD = raw_review_RDD_tmp.sample(False, 1/50, 0)
print(raw_review_RDD.count())

user_set = set(raw_review_RDD.map(lambda x: x[-1]).collect())
print(len(user_set))

139555


112956


In [7]:
#sample number of users and take all the reviews made by the users in the subset

raw_user_RDD = raw_user_RDD_tmp.filter(lambda x: (x[-2] in user_set))
raw_user_RDD.count()
#13608



112956

### Parse the RDD rows
Keep only the interested attributes and parse the rows into a tuple format. 

Hash the key values of type String into Integer values.

In [8]:
import hashlib

def hash(s):
    h = hashlib.sha1(s.encode())
    return int(h.hexdigest(),16)%((2**31)-1)

In [9]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (hash(str(entry["review_id"])),    # 0
            hash(str(entry["user_id"])),      # 1
            hash(str(entry["business_id"])),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')

#review_RDD.filter(lambda x: x[1]==1328793112).collect()

There are 139555 reviews in the dataset
Reviews: (2046062545, 1975347158, 361991568, 4, 1, 0, 1, "I was really between 3 and 4 stars for this one. I LOVE the 96th street Naked Tchopstix so I was very excited to see this one which is closer to my house.\nThe vibe is totally different as this is geared more to take out although they do have a decent size dining area. You order at the counter and they deliver it.\nMy daughter and I tried the sushi bowl. You pick up a piece of paper and  select from a choice of proteins, vegetables, rice and toppings. I like the fact that it is on paper and you don't have to guess what things are or have to shout over the counter to tell the server what you want. The sushi bowl was good but not wonderful. The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items. The sauces were tasty. My husband ordered the Maui roll which is my favorite roll at the 96th street location and this just was not u

In [10]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (hash(str(entry["user_id"])),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


There are 112956 users in the dataset
Users: (1515475869, 'Gwen', 224, 4.27, 512, 330, 299, 28)


In [11]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (hash(str(entry["business_id"])),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD_tmp.map(get_business_tuple)

business_count = business_RDD.count()
# print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

There are 150346 business in the dataset
Business: (1768608060, 'Abby Rappoport, LAC, CMQ', 'Santa Barbara', 'CA', 5.0, 7, ['Doctors', 'Traditional Chinese Medicine', 'Naturopathic/Holistic', 'Acupuncture', 'Health & Medical', 'Nutritionists'])


# Calculate overall
The variable review_RDD provides columns such as $useful$, $funny$ and $cool$. These fields lend us to a more accurate rating of the review, thus increasing (or decreasing) its star valuation and consequently the user consideration of the business. The item $fans$, belonging to the user dataset, is meaningful in terms of how much a reviewer could influence the opinion of someone interested in that particular business. These info are used for recalculating the ratings of each review, hence they should enhance their relevance. The decision of whether add or subtract $\Delta$, which is the value of how the review is significant, is determined by the number of the stars: if it is less than $3$, meaning that the business has not been appreciated by the user, the amount is decreased else viceversa.  The formula to determine $\Delta$ is the following:
### - for each business_id:

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{useful} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ useful} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [12]:
def get_business_id_useful_funny_cool(row):
    """
        Writes a tuple with key business_id and attributes useful, funny, cool
    """
    return (row[2], (row[4], row[5], row[6]))

def get_max_useful_funny_cool(row1, row2):
    """
        Returns a new RDD with, for each business_id key, the maxima values of useful, funny, cool
    """
    return tuple(max(row1[i], row2[i]) for i in range(3))

review_tuple_RDD = review_RDD.map(get_business_id_useful_funny_cool)
review_best_ufc_RDD = review_tuple_RDD.reduceByKey(get_max_useful_funny_cool)

review_best_ufc_RDD.take(5)

# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 


[(491225560, (3, 1, 1)),
 (1433893320, (20, 26, 2)),
 (1526625320, (0, 0, 0)),
 (813435520, (2, 0, 0)),
 (241544360, (0, 1, 0))]

In [13]:
def rearrange_review_and_best_ufc_RDD(row):
    """
        Merge, for each business_id, the row values with the best useful, funny, cool ones
    """
    return (row[2], (row[0], row[1], row[3], row[4], row[5], row[6]))

review_ufc_RDD = review_RDD.map(rearrange_review_and_best_ufc_RDD).join(review_best_ufc_RDD)
review_ufc_RDD.first()

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))


(813435520, ((803112855, 269845310, 4, 1, 0, 0), (2, 0, 0)))

In [14]:
def partial_overall(x):
    """
        Rearrange review_ufc_RDD and calculates the partial overall of the delta above
        partial_overall ={[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2]
    """
    key = (x[1][0][0], x[1][0][1], x[0], x[1][0][2])
    num = (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2)
    den = (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2)
    return (*key,(num/den if sum(x[1][1]) != 0 else 0))


partial_review_overall_RDD = review_ufc_RDD.map(partial_overall)
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

(803112855, 269845310, 813435520, 4, 0.5)

In [15]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


(1925401914, ((1416765511, 1172041840, 5, 1.0), 0))

In [16]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print("User with the highest number of fans: ",best_user_fans)


def review_overall(x):
    user_id, review_id, business_id, stars= x[0], x[1][0][0], x[1][0][1], x[1][0][2]
    value = stars + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if stars >=3 else -2) 
    return (review_id, user_id, business_id, value)

    #tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))


review_overall_RDD = partial_overall_fans_RDD.map(review_overall)

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(5)


User with the highest number of fans:  (526211061, 'Mike', 1882, 4.39, 22860, 10965, 15350, 12497)


[(2023460911, 652197590, 886219616, 5.639753540849804),
 (2020729754, 652197590, 1979569868, 5.639753540849804),
 (132956823, 789572584, 1443709662, 5.629751140273665),
 (415135109, 1417191503, 1792621891, 5.605105225254061),
 (1876462129, 1875221347, 2061717186, 5.595542930303273)]

# Basic Recommendation
We sort the ratings and we recommend the best businesses (with at least a total number of reviews).

In [17]:
#Keeping businesses that have at least 10 reviews
threshold = 10
review_by_business_filtered_RDD = review_overall_RDD.map(lambda x: (x[2], (x[0], x[1], x[3]))).groupByKey()\
                                                    .mapValues(list).filter(lambda x: len(x[1])>=threshold)
review_by_business_filtered_RDD.take(5)


[(1069254966,
  [(676866483, 953899088, 3.0002400576138273),
   (78350610, 1390056017, 5.000040009602305),
   (1811926675, 534268199, 3.000800192046091),
   (1999290740, 1871673068, 3.0380772866969554),
   (169254890, 835245666, 0.8144147187917693),
   (624444586, 91616234, 5.074074074074074),
   (951588744, 518574696, 3.03723708504856),
   (2049526450, 1485534045, 4.061516986298934),
   (1707489212, 1459213824, 5.0),
   (458506811, 1251762922, 1.0),
   (212813999, 1939676425, 3.3747314170215668),
   (1279446563, 1969239614, 5.037037037037037),
   (1582628910, 934591426, 3.150748772297944),
   (513916445, 2090234063, 4.000920220853005),
   (953857118, 1758721926, 2.0),
   (641019392, 1278364694, 5.0),
   (655936305, 919021832, 1.0),
   (1655129027, 824357901, 5.000040009602305),
   (1530901351, 1718417663, 1.0),
   (2082046852, 1865346809, 4.018558528120823),
   (258513375, 1924864859, 5.074234112483293),
   (62174369, 1477402290, 4.037077046639341),
   (1565038157, 1771082076, 5.00008

In [18]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """

    if (tuple[1][1]!= None):
        value = '{:.3f}'.format(tuple[1][1])
    else:
        value = ''
    key = tuple[0]
    return (value + ' ' + str(key))


In [19]:
#Make, for each business, the average of the stars for all the reviews

business_overall_RDD = review_by_business_filtered_RDD.map(lambda x: (x[0],  sum(i[2] for i in x[1])/len(x[1])))
business_overall_RDD.first()

(1069254966, 3.8521870248859713)

In [20]:
# top recommended businesses
business_RDD.map(lambda x: (x[0], (x[1], x[6]))).leftOuterJoin(business_overall_RDD).sortBy(sortFunction, False).take(10)  #x[2], x[3],.filter(lambda x: 'Restaurants' in x[1][1])


[(375888967,
  (('Maison De Cheveux',
    ['Beauty & Spas',
     'Makeup Artists',
     'Massage',
     'Nail Technicians',
     'Waxing',
     'Nail Salons',
     'Hair Removal',
     'Hair Salons']),
   5.118338219354463)),
 (1020789429,
  (('The Coronet',
    ['Bars',
     'Modern European',
     'Cocktail Bars',
     'Breakfast & Brunch',
     'Nightlife',
     'Cafes',
     'American (New)',
     'Brasseries',
     'Restaurants']),
   5.115495411206382)),
 (876118032,
  (('Tumerico',
    ['Mexican', 'Gluten-Free', 'Vegetarian', 'Restaurants', 'Vegan']),
   5.105583562277169)),
 (1485361545,
  (('Kaffe Crepe',
    ['Food', 'Restaurants', 'Cafes', 'Creperies', 'Coffee & Tea']),
   5.096504699589439)),
 (774249058,
  (('Blondes By Erica', ['Hair Salons', 'Beauty & Spas']), 5.090008001920461)),
 (674385531,
  (('Pickles & Swiss',
    ['Delis', 'Sandwiches', 'American (New)', 'Restaurants']),
   5.081709565046743)),
 (910287020,
  (('The Quesadillas', ['Tacos', 'Restaurants', 'Mexican'

# Collaborative Filtering
We now try to suggest business based on user's similarities. This criterion is measured by the Cosine Distance.

In [21]:
# review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))
review_overall_bis = review_RDD.map(lambda x: (x[1], x[2], x[3]))


In [22]:
#Cosine Distance
def cosine_dist(value):
    """ 
    arg: utente da confrontare
    per ogni utente:
        - user_id, prodotto tra rating di item per stessa chiave di item, lista di rating
        - somma di valori nel numeratore, somma di liste del denominatore
        - prodotto delle somma quadratiche nel denominatore tra i due utenti
        - prendi utenti che hanno qualche voto su un business in comune diverso da zero
    """
    value_review = dict(review_overall_bis.filter(lambda x:x[0] == value).map(lambda x:(x[1],x[2])).collect()) #business: voto
    print(value_review)
    if (len(value_review)==0): 
        return None, None
    user_sim = review_overall_bis.map(lambda x: (x[0], (0 if x[1] not in value_review.keys() \
                                                        else x[2]*value_review[x[1]], [x[2]])))\
                                 .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                                 .map(lambda x:(x[0], x[1][0]/((sum(i**2 for i in x[1][1])**0.5) * \
                                                               (sum(i**2 for i in value_review.values())**0.5))))\
                                 .filter(lambda x:x[1]!=0 and x[0]!=value).take(5) #prendiamo i 5 utenti più simili
    return sorted(user_sim), value_review

def collaborative_filtering(value):
    user_sim, value_review = cosine_dist(value)
    if (user_sim == None and value_review == None):
        print("Unable to do the Collab Filtering for this user")
        return
    return review_overall_bis.filter(lambda x: (x[0] in dict(user_sim).keys() and x[1] not in value_review.keys()))\
                             .take(10) #.sortBy(lambda x:x[2],ascending = False)


In [23]:
# print(review_overall_bis.take(10)[5])
business_collab = collaborative_filtering(652197590) # 79992688
# print(review_overall_RDD.filter(lambda x:x[1]=='Eep1pCr1zMqub16tYgUFgQ').collect())
business_collab
#TODO make print of the business as name, categories...

{1979569868: 5, 2084822936: 3, 1597357753: 3, 886219616: 5, 930963096: 4}


[]

In [24]:
# recommended_business = sorted(business_collab, key=lambda x : -x[2])
business_RDD.filter(lambda x: x[0] in [i[1] for i in business_collab]).collect()

[]

# UV Decompostion
This approach predicts the blank values of the utility matrix by multiplying two matrices, U and V.

# Initialization

In [61]:
#We need to train and predict the vectors U and V
# train(trainingRDD, rank, iterations, regularization_parameter)  --> (U,V)
#We need to calculate RMSE between the utility and our matrix

UV_rank = 2
# regularization_parameter = 0.1

U_RDD = user_RDD.map(lambda x: (x[0], [1 for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [1 for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])



In [62]:
#NOTE We are bypassing at the moment the validation set (useful for achieving the best rank)
trainingRDD, testRDD = review_overall_bis.randomSplit([7,3], seed=0)

Broadcast the data to the workers.

In [63]:
M_BC = sc.broadcast(trainingRDD.collect())


In [64]:
V_BC = sc.broadcast(dict(V_RDD.collect()))


In [65]:
U_BC = sc.broadcast(dict(U_RDD.collect()))


# Training 

In [66]:
# normalized-->tuple: (user_id, business_id, overall - normalized_value)
# review_overall --> tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))

def update_U(entry):
    id = entry[0]
    U_value = entry[1]
    reviews = [i for i in M_BC.value if i[0] == id]
    for s in range(UV_rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[1] # business_id
            p = 0
            for k in [i for i in range(UV_rank) if i != s]:                
                p += U_value[k] * V_BC.value[j][k]

            res += V_BC.value[j][s] * (review[2] - p)
            den += V_BC.value[j][s]**2
        if res!=0 and den!=0:
            U_value[s] = (res) / (den)

    return (id, U_value)


In [67]:
def update_V(entry):
    id = entry[0]
    V_value = entry[1]
    reviews = [i for i in M_BC.value if i[1] == id]

    for s in range(UV_rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[0] # user_id
            p = 0
            for k in [i for i in range(UV_rank) if i != s]:
                p += V_value[k] * U_BC.value[j][k]
                
            res += U_BC.value[j][s] * (review[2] - p)
            den += U_BC.value[j][s]**2
        if res != 0 and den!=0:
            V_value[s] = ((res)/ (den))

    return (id, V_value)


In [68]:
def train(U_RDD,V_RDD,iterations):  
# iterations = 2
    global V_BC, U_BC
    for i in range(iterations):
        V_BC = sc.broadcast(dict(V_RDD.collect()))
        U_RDD = U_RDD.map(update_U)

        U_BC = sc.broadcast(dict(U_RDD.collect()))
        V_RDD = V_RDD.map(update_V)

    return U_RDD.collect(), V_RDD.collect()
    

This cell is the most time consuming... for this reason we have sampled the dataset.

In [69]:
U,V = train(U_RDD, V_RDD, iterations =2)

U and V matrices

In [70]:
U

[(1515475869, [1, 1]),
 (1804702248, [1, 1]),
 (67330113, [3.2962219830314554, 0.9044397912668775]),
 (73769501, [1, 1]),
 (1429843609, [1, 1]),
 (1182890019, [1, 1]),
 (1062844705, [1.8720000000000003, 0.9999999999999998]),
 (785048578, [2.3826671830887047, 1.013029667004431]),
 (115560766, [2.219309346026608, 0.9236697385845763]),
 (1788593616, [4.044496046010065, 1.0000000000000004]),
 (580812163, [3.124855324074075, 0.9999999999999989]),
 (2127759282, [1, 1]),
 (253031104, [1.1894530862036363, 1.3589854620683373]),
 (597959812, [3.98375, 1.0]),
 (905246111, [1, 1]),
 (751461856, [3.0, 1.0]),
 (478021564, [3.22569416828242, 0.9805621974398291]),
 (1836485906, [3.0430830039525696, 0.9999999999999993]),
 (1607055882, [3.848644163302047, 0.9455866057972345]),
 (1368183967, [2.082142857142857, 1.0000000000000002]),
 (1088909204, [4.0, 1.0]),
 (1297302066, [3.9062775317039975, 1.001058646185522]),
 (1417191503, [3.7501443176350544, 0.9660180416955613]),
 (166020942, [3.998314369995786, 0

In [71]:
V

[(1768608060, [1, 1]),
 (674023133, [1, 1]),
 (1853240994, [1, 1]),
 (25885286, [1.0, 1.0]),
 (886402536, [1.1158521742609802, 1.0554441525714118]),
 (899415875, [1, 1]),
 (151330391, [1, 1]),
 (659900618, [1, 1]),
 (2073408338, [1, 1]),
 (259476066, [1, 1]),
 (1270265292, [1, 1]),
 (901865183, [1, 1]),
 (399025230, [1, 1]),
 (1188362622, [1, 1]),
 (1652935626, [1, 1]),
 (905182201, [1.0, 1.0]),
 (464645059, [1, 1]),
 (1007062082, [1, 1]),
 (1721829350, [1, 1]),
 (1979569868, [1.0, 1.0]),
 (464040412, [1, 1]),
 (904870512, [1, 1]),
 (464545888, [1.0, 1.0]),
 (358680582, [1.0018443862105046, 0.976223141433311]),
 (395093298, [1, 1]),
 (2071797272, [1, 1]),
 (1451936390, [1, 1]),
 (330641765, [1.0, 1.0]),
 (1630810918, [1, 1]),
 (1725190102, [1, 1]),
 (1689700950, [1.0, 1.0]),
 (701452773, [1, 1]),
 (1941251565, [1, 1]),
 (271351505, [1, 1]),
 (2107667707, [1, 1]),
 (749861318, [0.8455758249254766, 1.0]),
 (786392692, [2.586956521739131, 0.999999999999999]),
 (503111654, [1, 1]),
 (19018

## Prediction and test of the UV model

In [72]:
#Take an user-id and suggest business that he has not reviewed

def predict(u_value, business_value):
    return sum(u_value[i]* business_value[i] for i in range(len(u_value)))


def print_prediction(username, business_name):

    U_dict = dict(U)
    print(U_dict[username])
    V_dict = dict(V)
    print(V_dict[business_name])
    un = user_RDD.filter(lambda x: x[0]== username).first()[1]
    bn = business_RDD.filter(lambda x: x[0]== business_name).first()[2]
    print("User:",un, "----- Business:",bn, ":",predict(U_dict[username], V_dict[business_name]))



In [73]:
#RMSE of UV Decomposition
import math 
def UV_rmse(testRDD):
    rmse = 0
    U_dict = dict(U)
    V_dict = dict(V)
    test = testRDD.collect()
    a = test
    for i in a:
        p = predict(U_dict[i[0]], V_dict[i[1]])
        rmse += (i[2] - p)**2
    return math.sqrt(rmse/len(a))


UV_loss = UV_rmse(testRDD)
print("RMSE:", UV_loss)

#user: 1125166186
#business: 1377779591



RMSE: 2.1608435090426905


In [74]:
print_prediction(1125166186, 1377779591)
print(U_RDD.filter(lambda x: x[1]))

print(review_overall_bis.filter(lambda x: x[0]==1125166186).collect())
print(review_overall_bis.filter(lambda x: x[1]==1377779591).collect())

[1, 1]
[1, 1]


User: Johna ----- Business: Indian Rocks Beach : 2
PythonRDD[699] at RDD at PythonRDD.scala:53


[(1125166186, 1377779591, 5), (1125166186, 1971412564, 1)]


[(1125166186, 1377779591, 5)]


Try to populate U and V with the average value of review rating

In [75]:
avg_value = review_overall_bis.map(lambda x: (1, x[2])).reduceByKey(lambda x,y:(x+y)/2).first()[1]
avg_value

4.215477358022409

In [76]:
U_RDD = user_RDD.map(lambda x: (x[0], [math.sqrt(avg_value/UV_rank) for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [math.sqrt(avg_value/UV_rank) for _ in range(UV_rank)]))#.sortBy(lambda x: x[0])"""


In [77]:
U,V = train(U_RDD, V_RDD, iterations =2)

In [78]:
U

[(1515475869, [1.4518053171865726, 1.4518053171865726]),
 (1804702248, [1.4518053171865726, 1.4518053171865726]),
 (67330113, [1.4650479931498184, 1.4115438459825818]),
 (73769501, [1.4518053171865726, 1.4518053171865726]),
 (1429843609, [1.4518053171865726, 1.4518053171865726]),
 (1182890019, [1.4518053171865726, 1.4518053171865726]),
 (1062844705, [0.5485886208035008, 1.4518053171865726]),
 (785048578, [0.8297035120581504, 1.440200982220385]),
 (115560766, [0.8609230272059316, 1.3168058623507723]),
 (1788593616, [2.02559239031577, 1.4518053171865724]),
 (580812163, [1.276704893025984, 1.4518053171865726]),
 (2127759282, [1.4518053171865726, 1.4518053171865726]),
 (253031104, [-0.034083421367647825, 1.581487114880145]),
 (597959812, [1.8994963353915788, 1.4518053171865721]),
 (905246111, [1.4518053171865726, 1.4518053171865726]),
 (751461856, [1.3033850328195347, 1.4518053171865726]),
 (478021564, [1.4612735996281776, 1.4311402249989873]),
 (1836485906, [1.331035557376983, 1.451805317

In [79]:
V

[(1768608060, [1.4518053171865726, 1.4518053171865726]),
 (674023133, [1.4518053171865726, 1.4518053171865726]),
 (1853240994, [1.4518053171865726, 1.4518053171865726]),
 (25885286, [1.4518053171865726, 1.4518053171865726]),
 (886402536, [1.6764558532534273, 1.583530721761575]),
 (899415875, [1.4518053171865726, 1.4518053171865726]),
 (151330391, [1.4518053171865726, 1.4518053171865726]),
 (659900618, [1.4518053171865726, 1.4518053171865726]),
 (2073408338, [1.4518053171865726, 1.4518053171865726]),
 (259476066, [1.4518053171865726, 1.4518053171865726]),
 (1270265292, [1.4518053171865726, 1.4518053171865726]),
 (901865183, [1.4518053171865726, 1.4518053171865726]),
 (399025230, [1.4518053171865726, 1.4518053171865726]),
 (1188362622, [1.4518053171865726, 1.4518053171865726]),
 (1652935626, [1.4518053171865726, 1.4518053171865726]),
 (905182201, [1.451805317186572, 1.4518053171865735]),
 (464645059, [1.4518053171865726, 1.4518053171865726]),
 (1007062082, [1.4518053171865726, 1.45180531

In [80]:
#RMSE of UV Decomposition
UV_avg_loss = UV_rmse(testRDD)
print("RMSE:", UV_avg_loss)


RMSE: 1.8668374465523605


# ALS

Compare UV Model with ALS model (using the implementation of movie_recommender.ipynb)

In [81]:
trainingRDDALS, validationRDDALS, testRDDALS = review_overall_bis.randomSplit([6, 2, 2], seed=0)

print('Training: %s, validation: %s, test: %s\n' % (trainingRDDALS.count(),
                                                    validationRDDALS.count(),
                                                    testRDDALS.count()))
print(trainingRDDALS.take(3))
print(validationRDDALS.take(3))
print(testRDDALS.take(3))

Training: 83708, validation: 27712, test: 28135

[(1975347158, 361991568, 4), (1675473854, 618074839, 2), (471527317, 446559988, 5)]
[(1450957654, 32178879, 5), (1212702022, 772045630, 5), (206699454, 1695023353, 4)]
[(385879061, 684052829, 3), (1313691670, 1859023395, 5), (821184238, 380457442, 1)]


In [82]:
import math

def computeError(predictedRDD, actualRDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictedRDD: predicted ratings for each movie and each user where each entry is in the form
                      (UserID, BusinessID, Rating)
        actualRDD: actual ratings where each entry is in the form (UserID, BusinessID, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictedRDD into the tuples of the form ((UserID, BusinessID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Transform actualRDD into the tuples of the form ((UserID, BusinessID), Rating)
    actualReformattedRDD = actualRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Business ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squaredErrorsRDD = (predictedReformattedRDD
                        .join(actualReformattedRDD)
                        .map(lambda i: math.pow(i[1][0] - i[1][1], 2))
                       )

    # Compute the total squared error - do not use collect()
    totalError = squaredErrorsRDD.reduce(lambda a, b: a+b)

    # Count the number of entries for which you computed the total squared error
    numRatings = squaredErrorsRDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.pow(float(totalError) / numRatings, 0.5)

In [86]:
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDDALS.map(lambda i: (i[0], i[1]))

iterations = 2 # 5
regularizationParameter = 0.1
ranks = [2, 4, 8, 12]
errors = [0, 0, 0, 0]
err = 0
tolerance = 0.02

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDDALS, rank, iterations=iterations,
                      lambda_=regularizationParameter, seed=0)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDDALS)
    errors[err] = error
    err += 1
    print('For rank {} the RMSE is {}'.format(rank, error))
    if error < minError:
        minError = error
        bestRank = rank

print('The best model was trained with rank {}'.format(bestRank))

For rank 2 the RMSE is 5.958020611182166


For rank 4 the RMSE is 4.9021514362722405


For rank 8 the RMSE is 4.450190980995117


For rank 12 the RMSE is 4.293743252770519
The best model was trained with rank 12


In [87]:
myModel = ALS.train(trainingRDDALS, bestRank, iterations=iterations,
                    lambda_=regularizationParameter)
testForPredictingRDDALS = testRDDALS.map(lambda i: (i[0], i[1]))
predictedTestRDDALS = myModel.predictAll(testForPredictingRDDALS)

testRMSE = computeError(testRDDALS, predictedTestRDDALS)

print('The model had a RMSE on the test set of {}'.format(testRMSE))

The model had a RMSE on the test set of 4.252192870389023


## ALS vs UV

Comparing the UV Decoposition implemented from Scratch against Spark's ALS (with rank 2).

In [88]:
model = ALS.train(trainingRDD, 2, iterations=2,
                      lambda_=regularizationParameter)
testForPredictingRDD = testRDD.map(lambda i: (i[0], i[1]))
predictedRatingsRDD = model.predictAll(testForPredictingRDD)

print("RMSE of ALS: ", computeError(testRDD, predictedRatingsRDD))
print("RMSE of UV with entries equal to 1: ", UV_loss)
print("RMSE of UV with entries referring to the average value : ", UV_avg_loss)

RMSE of ALS:  5.879441546413611
RMSE of UV with entries equal to 1:  2.1608435090426905
RMSE of UV with entries referring to the average value :  1.8668374465523605
